In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import ipywidgets as widgets
from ipywidgets import interactive
from IPython.display import display
# from scipy.signal import find_peaks # <<< ไม่ต้องใช้แล้ว (ถูกต้อง!)

# --- ส่วนที่ 1: ตั้งค่า (เวอร์ชั่นแก้ไข) ---

# 1. ย้ายค่าคงที่ทั้งหมดออกมานอกลูป!
#    การประกาศลิสต์ยักษ์พวกนี้ซ้ำๆ 50 รอบมันเปลืองทรัพยากร
MEM_MAX_FRAMES = [480, 600, 720, 840, 960, 1080, 1200, 1320, 1440, 1560]
MEM_tick = ["1tick", "2tick", "3tick", "4tick", "5tick", "6tick", "7tick", "8tick", "9tick", "10tick"]
MEM_TARGET_FILE_NAME = [
    "1tick_1.npy", "1tick_2.npy", "1tick_3.npy", "1tick_4.npy", "1tick_5.npy",
    "2tick_1.npy", "2tick_2.npy", "2tick_3.npy", "2tick_4.npy", "2tick_5.npy",
    "3tick_1.npy", "3tick_2.npy", "3tick_3.npy", "3tick_4.npy", "3tick_5.npy",
    "4tick_1.npy", "4tick_2.npy", "4tick_3.npy", "4tick_4.npy", "4tick_5.npy",
    "5tick_1.npy", "5tick_2.npy", "5tick_3.npy", "5tick_4.npy", "5tick_5.npy",
    "6tick_1.npy", "6tick_2.npy", "6tick_3.npy", "6tick_4.npy", "6tick_5.npy",
    "7all_1.npy", "7tick_2.npy", "7tick_3.npy", "7tick_4.npy", "7tick_5.npy",
    "8tick_1.npy", "8tick_2.npy", "8tick_3.npy", "8tick_4.npy", "8tick_5.npy",
    "9tick_1.npy", "9tick_2.npy", "9tick_3.npy", "9tick_4.npy", "9tick_5.npy",
    "10tick_1.npy", "10tick_2.npy", "10tick_3.npy", "10tick_4.npy", "10tick_5.npy"
]
base_folder_path = r"D:\TEMP_DOWNLOAD\31_10_25_precise\31_10_25_precise"

# 2. ใช้ FOR LOOPS ที่ถูกต้อง
#    ลูป 'ii' (0-9) สำหรับแต่ละ "tick"
#    ลูป 'jj' (0-4) สำหรับแต่ละ "ไฟล์ย่อย"
for ii in range(10):  # ii จะวนจาก 0 ถึง 9
    for jj in range(5):  # jj จะวนจาก 0 ถึง 4

        # 3. คำนวณค่าสำหรับรอบนี้
        file_index = (ii * 5) + jj  # <--- นี่คือการแก้บั๊ก [ii][jj]
        
        folder_path = os.path.join(base_folder_path, MEM_tick[ii])
        TARGET_FILE_NAME = MEM_TARGET_FILE_NAME[file_index]
        MAX_FRAMES_TO_SHOW = MEM_MAX_FRAMES[ii]

        # --- ส่วนที่ 2: โหลดไฟล์เป้าหมาย (เหมือนเดิม) ---
        file_path = os.path.join(folder_path, TARGET_FILE_NAME)
        print(f"=====================================================")
        print(f"กำลังโหลดไฟล์เป้าหมาย: {file_path}")
        print(f"=====================================================")

        try:
            if not os.path.exists(file_path):
                raise FileNotFoundError(f"หาไฟล์ไม่เจอ: {file_path}")

            img_data_full = np.load(file_path)
            print(f"โหลดสำเร็จ! Shape ของข้อมูล: {img_data_full.shape}")

            # --- ส่วนที่ 3: ปรับปรุงใหม่! (Phase Unwrapping) ---
            # (โค้ดส่วนนี้ของคุณดีอยู่แล้ว ผมแค่ย้ายมันเข้ามาใน loop ที่ถูกต้อง)

            if img_data_full.ndim == 4:
                
                total_actual_frames = img_data_full.shape[0]
                num_frames = min(total_actual_frames, MAX_FRAMES_TO_SHOW)
                
                if num_frames < 2: # ต้องมีอย่างน้อย 2 เฟรม
                    print("เฟรมน้อยเกินไป (ต้องการอย่างน้อย 2) ไม่สามารถวิเคราะห์ Phase ได้")
                    continue # ข้ามไปไฟล์ถัดไป

                img_data = img_data_full[0:num_frames] 
                
                # --- เตรียมข้อมูล Grayscale ---
                if img_data.shape[3] > 1:
                    print("ตรวจพบรูปสี กำลังแปลงเป็น Grayscale เพื่อวิเคราะห์...")
                    gray_data = np.mean(img_data, axis=3) 
                else:
                    gray_data = img_data.squeeze(axis=3)
                
                print(f"ข้อมูลสำหรับวิเคราะห์ (Frames, H, W): {gray_data.shape}")
                print(f"จะแสดงผลและวิเคราะห์ {num_frames} เฟรมแรก")

                # --- 1. สร้าง SLIDERS ---
                frame_slider = widgets.IntSlider(
                    value=0, min=0, max=num_frames - 1, step=1,
                    description=f'{TARGET_FILE_NAME} Frame:',
                    layout=widgets.Layout(width='80%')
                )
                y_slider = widgets.IntSlider(
                    value=gray_data.shape[1] // 2, min=0, max=gray_data.shape[1] - 1, step=1,
                    description='Y-Coord (จุด I):',
                    layout=widgets.Layout(width='80%')
                )
                x_slider = widgets.IntSlider(
                    value=gray_data.shape[2] // 2, min=0, max=gray_data.shape[2] - 1, step=1,
                    description='X-Coord (จุด I):',
                    layout=widgets.Layout(width='80%')
                )
                dy_slider = widgets.IntSlider(
                    value=0, min=-50, max=50, step=1,
                    description='Offset Y (สำหรับจุด Q):',
                    layout=widgets.Layout(width='80%')
                )
                dx_slider = widgets.IntSlider(
                    value=5, min=-50, max=50, step=1, # เริ่มที่ 5 จะได้ไม่ซ้ำจุด I
                    description='Offset X (สำหรับจุด Q):',
                    layout=widgets.Layout(width='80%')
                )

                # --- 2. สร้างฟังก์ชันวาดรูป + วิเคราะห์ Phase ---
                # (*** ไม่ต้องเปลี่ยนอะไรในฟังก์ชันนี้เลย ***)
                def plot_phase_analysis(frame_idx, y_coord, x_coord, dy, dx, local_gray_data=gray_data):
                    # หมายเหตุ: ผมเพิ่ม local_gray_data เข้าไปใน argument
                    # เพื่อ "ล็อค" gray_data ของไฟล์ปัจจุบันไว้
                    # ไม่งั้น slider ของไฟล์เก่าจะไปเรียกใช้ gray_data ของไฟล์ใหม่
                    
                    # --- ส่วนที่ 1: คำนวณ Phase ---
                    y_q = np.clip(y_coord + dy, 0, local_gray_data.shape[1] - 1)
                    x_q = np.clip(x_coord + dx, 0, local_gray_data.shape[2] - 1)

                    signal_I = local_gray_data[:, y_coord, x_coord]
                    signal_Q = local_gray_data[:, y_q, x_q]
                    
                    sig_I_proc = signal_I - np.mean(signal_I)
                    sig_Q_proc = signal_Q - np.mean(signal_Q)

                    wrapped_phase = np.arctan2(sig_Q_proc, sig_I_proc)
                    unwrapped_phase = np.unwrap(wrapped_phase) # นี่คือ "บริการช่วยจำ"!

                    total_phase_change = unwrapped_phase[-1] - unwrapped_phase[0]
                    total_net_fringes = total_phase_change / (2 * np.pi)
                    
                    current_phase_change = unwrapped_phase[frame_idx] - unwrapped_phase[0]
                    current_net_fringes = current_phase_change / (2 * np.pi)

                    # --- ส่วนที่ 2: วาดกราฟ (อัปเกรดเป็น 3 ช่อง) ---
                    fig = plt.figure(figsize=(16, 12))
                    
                    ax1 = plt.subplot(2, 2, 1)
                    ax1.imshow(local_gray_data[frame_idx], cmap='gray')
                    ax1.plot(x_coord, y_coord, 'r+', markersize=15, label=f'Point I ({y_coord}, {x_coord})')
                    ax1.plot(x_q, y_q, 'bx', markersize=15, label=f'Point Q ({y_q}, {x_q})')
                    ax1.legend()
                    ax1.set_title(f"Frame {frame_idx}")
                    ax1.axis('off')

                    ax2 = plt.subplot(2, 2, 2)
                    ax2.plot(sig_I_proc, 'r-', label='Signal I (DC Removed)')
                    ax2.plot(sig_Q_proc, 'b-', label='Signal Q (DC Removed)')
                    ax2.axvline(frame_idx, color='k', linestyle='--', alpha=0.5)
                    ax2.set_title("I/Q Signals (พยายามจูนให้ห่างกัน 90 องศา)")
                    ax2.set_xlabel("Frame Index")
                    ax2.set_ylabel("Intensity (Processed)")
                    ax2.legend()
                    
                    ax3 = plt.subplot(2, 1, 2)
                    ax3.plot(unwrapped_phase, 'g-', label='Unwrapped Phase (Radians)')
                    ax3.axvline(frame_idx, color='k', linestyle='--', alpha=0.5, label=f'Current Frame: {frame_idx}')
                    
                    start_phase = unwrapped_phase[0]
                    # แก้ไขเล็กน้อย:
                    min_fringe = int(np.floor(np.min(unwrapped_phase - start_phase) / (2 * np.pi)))
                    max_fringe = int(np.ceil(np.max(unwrapped_phase - start_phase) / (2 * np.pi)))
                    
                    for i in range(min_fringe, max_fringe + 1):
                        if i == 0: continue
                        ax3.axhline(start_phase + i * (2 * np.pi), color='k', linestyle=':', alpha=0.3)

                    ax3.set_title(f"Phase 'Memory' | Current Net Fringes: {current_net_fringes:.2f} | Total Net Fringes: {total_net_fringes:.2f}")
                    ax3.set_xlabel("Frame Index")
                    ax3.set_ylabel("Total Phase (Radians)")
                    ax3.legend()
                    ax3.grid(True, linestyle=':')

                    plt.tight_layout()
                    plt.show()
                
                # --- 3. "มัด" Slider ทั้งหมดเข้ากับฟังก์ชัน ---
                output = interactive(plot_phase_analysis, 
                                     frame_idx=frame_slider, 
                                     y_coord=y_slider, 
                                     x_coord=x_slider,
                                     dy=dy_slider,
                                     dx=dx_slider,
                                     local_gray_data=widgets.fixed(gray_data)) # "ล็อค" gray_data ไว้
                
                # 4. แสดงผล (จะแสดงผล 1 widget ต่อ 1 ไฟล์)
                print(f"--- แสดงผล Interactive Plot สำหรับ {TARGET_FILE_NAME} ---")
                display(output)
                
            # เคสที่เป็นรูปเดี่ยว (เหมือนเดิม)
            elif img_data_full.ndim == 3 or img_data_full.ndim == 2:
                print("ไฟล์นี้เป็นภาพเดี่ยว (ไม่สามารถนับ fringe ข้ามเวลาได้) กำลังแสดงผล...")
                plt.figure(figsize=(8, 6))
                if img_data_full.ndim == 3:
                    plt.imshow(img_data_full, cmap='gray')
                else: # ndim == 2
                    plt.imshow(img_data_full, cmap='gray')
                plt.title(f"Single Image: {TARGET_FILE_NAME}")
                plt.axis('off')
                plt.show()
            
            else:
                print(f"มิติข้อมูลแปลกๆ ไม่สามารถแสดงผลได้: {img_data_full.shape}")

        except Exception as e:
            print(f"--- เกิดข้อผิดพลาด ---")
            print(f"Error: {e}")
            # continue # ถ้าอยากให้มันรันต่อแม้จะเจอปัญหาก็เปิดคอมเมนต์นี้

print("\n--- DONE ---")
print("ประมวลผลไฟล์ทั้งหมดเสร็จสิ้น")

กำลังโหลดไฟล์เป้าหมาย: D:\TEMP_DOWNLOAD\31_10_25_precise\31_10_25_precise\1tick\1tick_1.npy
โหลดสำเร็จ! Shape ของข้อมูล: (480, 200, 200, 3)
ตรวจพบรูปสี กำลังแปลงเป็น Grayscale เพื่อวิเคราะห์...
ข้อมูลสำหรับวิเคราะห์ (Frames, H, W): (480, 200, 200)
จะแสดงผลและวิเคราะห์ 480 เฟรมแรก


--- แสดงผล Interactive Plot สำหรับ 1tick_1.npy ---


interactive(children=(IntSlider(value=0, description='1tick_1.npy Frame:', layout=Layout(width='80%'), max=479…

กำลังโหลดไฟล์เป้าหมาย: D:\TEMP_DOWNLOAD\31_10_25_precise\31_10_25_precise\1tick\1tick_2.npy
โหลดสำเร็จ! Shape ของข้อมูล: (480, 200, 200, 3)
ตรวจพบรูปสี กำลังแปลงเป็น Grayscale เพื่อวิเคราะห์...
ข้อมูลสำหรับวิเคราะห์ (Frames, H, W): (480, 200, 200)
จะแสดงผลและวิเคราะห์ 480 เฟรมแรก


--- แสดงผล Interactive Plot สำหรับ 1tick_2.npy ---


...

interactive(children=(IntSlider(value=0, description='10tick_4.npy Frame:', layout=Layout(width='80%'), max=15…

กำลังโหลดไฟล์เป้าหมาย: D:\TEMP_DOWNLOAD\31_10_25_precise\31_10_25_precise\10tick\10tick_5.npy
โหลดสำเร็จ! Shape ของข้อมูล: (1560, 200, 200, 3)
ตรวจพบรูปสี กำลังแปลงเป็น Grayscale เพื่อวิเคราะห์...
ข้อมูลสำหรับวิเคราะห์ (Frames, H, W): (1560, 200, 200)
จะแสดงผลและวิเคราะห์ 1560 เฟรมแรก


--- แสดงผล Interactive Plot สำหรับ 10tick_5.npy ---


interactive(children=(IntSlider(value=0, description='10tick_5.npy Frame:', layout=Layout(width='80%'), max=15…


--- DONE ---
ประมวลผลไฟล์ทั้งหมดเสร็จสิ้น
